In [1]:
import json
import time
from pypdf import PdfReader


In [2]:
def read_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

In [3]:
summary_text = open("me/business_summary.txt.txt", "r", encoding="utf-8").read()
pdf_text = read_pdf("me/about_business.pdf.pdf")


In [4]:
def record_customer_interest(email, name, message):
    lead = {
        "time": time.time(),
        "email": email,
        "name": name,
        "message": message
    }
    with open("leads.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(lead, ensure_ascii=False) + "\n")
    return "Lead saved."


def record_feedback(question):
    fb = {
        "time": time.time(),
        "question": question
    }
    with open("feedback.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(fb, ensure_ascii=False) + "\n")
    return "Feedback saved."


In [5]:
def record_customer_interest(email, name, message):
    lead = {
        "time": time.time(),
        "email": email,
        "name": name,
        "message": message
    }
    with open("leads.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(lead, ensure_ascii=False) + "\n")
    return "Lead saved."


def record_feedback(question):
    fb = {
        "time": time.time(),
        "question": question
    }
    with open("feedback.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(fb, ensure_ascii=False) + "\n")
    return "Feedback saved."


In [6]:
import re

def extract_email(text):
    m = re.search(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", text)
    return m.group(0) if m else None

# simple state to remember email until user gives name + message
state = {"pending_email": None}

def bot_reply(message, history):
    msg = message.strip()
    email = extract_email(msg)

    # Step A: user sends email
    if email:
        state["pending_email"] = email
        return "Thanks! Please tell me your name and what you need (one short sentence)."

    # Step B: user sends name + need after email
    if state["pending_email"]:
        # very simple parsing: first word as name
        name = msg.split()[0] if msg.split() else "Unknown"
        record_customer_interest(
            email=state["pending_email"],
            name=name,
            message=msg
        )
        state["pending_email"] = None
        return "Perfect ✅ I recorded your interest. Anything else you want to ask about StudyMate AI?"

    # Step C: If question is not answered by our info, record feedback (simple heuristic)
    # We'll treat unrelated topics as feedback
    unrelated = ["bitcoin", "gold", "football", "movie", "weather", "stock", "price of"]
    if any(k in msg.lower() for k in unrelated):
        record_feedback(msg)
        return "I’m not sure about that, but I saved your question as feedback. Do you have a question about StudyMate AI?"

    # Step D: Default company response (uses your info)
    return (
        "Hi! I’m the StudyMate AI assistant. "
        "We help university students with clear explanations, summaries, and practice questions. "
        "Ask me about our services or mission, or share your email if you want to join."
    )


In [7]:
import gradio as gr
gr.ChatInterface(bot_reply).launch()


c:\Users\Dell 3400\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
